In [2]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [3]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "openssl", 26

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'crypto/x509/x509_vpm.c'}
bug.buggy_lines={('crypto/x509/x509_vpm.c', 212), ('crypto/x509/x509_vpm.c', 201), ('crypto/x509/x509_vpm.c', 202)}
bug.coverage_df=


79  \
file                                                                      function                    line       
/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c   EC_KEY_new                  31    0    
                                                                                                      33    0    
                                                                          EC_KEY_new_ex               37    0    
                                                                                                      39    0    
                                                                          EC_KEY_new_by_curve_name_ex 42    0    
...                                                                                                        ..    
/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c EVP_MD_CTX_clear_flags      987   0    
                                                                                                      988   0    
                                                                          EVP_CIPHER_CTX_set_flags    995   0    
                                                                                                      997   0    
                                                                                                      998   0    

                                                                                                            146  \
file                                                                      function                    line        
/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c   EC_KEY_new                  31    0     
                                                                                                      33    0     
                                                                          EC_KEY_new_ex               37    0     
                                                                                                      39    0     
                                                                          EC_KEY_new_by_curve_name_ex 42    0     
...                                                                                                        ..     
/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c EVP_MD_CTX_clear_flags      987   0     
                                                                                                      988   0     
                                                                          EVP_CIPHER_CTX_set_flags    995   0     
                                                                                                      997   0     
                                                                                                      998   0     

                                                                                                            134  \
file                                                                      function                    line        
/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c   EC_KEY_new                  31    0     
                                                                                                      33    0     
                                                                          EC_KEY_new_ex               37    5     
                                                                                                      39    5     
                                                                          EC_KEY_new_by_curve_name_ex 42    0     
...                                                                                                        ..     
/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c EVP_MD_CTX_clear_flags      987   20    
                                                                                              

bug.failing_tests={'207'}
bug.failing_tests_GT={'207'}
bug.functions={('/home/workspace/ssl/libssl-shlib-ssl_conf.gcno//ssl/ssl_conf.c', 'ssl_set_option'), ('/home/workspace/crypto/crmf/libcrypto-lib-crmf_lib.gcno//crypto/crmf/crmf_lib.c', 'OSSL_CRMF_CERTTEMPLATE_get0_serialNumber'), ('/home/workspace/apps/lib/libapps-lib-apps.gcno//apps/lib/apps.c', 'pkey_ctrl_string'), ('/home/workspace/apps/openssl-bin-fipsinstall.gcno//apps/fipsinstall.c', 'print_mac'), ('/home/workspace/crypto/ec/libcrypto-lib-ec_ameth.gcno//crypto/ec/ec_ameth.c', 'eckey_pub_encode'), , ...}
bug.id='openssl-26'
bug.num_test_cases=233
bug.passing_tests={'108', '79', '75', '201', '66', '38', '162', '130', '141', '140', '6', '49', '157', '194', '203', '214', '233', '39', '177', '84', '174', '101', '61', '48', '105', '45', '199', '97', '169', '191', '173', '88', '205', '50', '15', '182', '175', '223', '127', '5', '180', '183', '8', '68', '46', '99', '170', '213', '3', '133', '40', '1', '93', '155', '60', '200', '90', 

# Get spectra

In [4]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                                                      function                    line                                                                                                               
/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c   EC_KEY_new                  31    (/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new, 31)                    
                                                                                                      33    (/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new, 33)                    
                                                                          EC_KEY_new_ex               37    (/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_ex, 37)                 
                                                                                                      39    (/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_ex, 39)                 
                                                                          EC_KEY_new_by_curve_name_ex 42    (/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_by_curve_name_ex, 42)   
...                                                                                                                                                                                                  ...                 
/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c EVP_MD_CTX_clear_flags      987   (/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_MD_CTX_clear_flags, 987)     
                                                                                                      988   (/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_MD_CTX_clear_flags, 988)     
                                                                          EVP_CIPHER_CTX_set_flags    995   (/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_CIPHER_CTX_set_flags, 995)   
                                                                                                      997   (/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_CIPHER_CTX_set_flags, 997)   
                                                                                                      998   (/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_CIPHER_CTX_set_flags, 998)   

                                                                                                            e_p  \
file                                                                      function                    line        
/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c   EC_KEY_new                  31    6     
                                                                                                      33    6     
                                                                          EC_KEY_new_ex               37    12    
                                                                                                      39    12    
                                                                          EC_KEY_new_by_curve_name_ex 42    3     
...                                                                                                         ..    
/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c EVP_MD_CTX_clear_flags      987   20    
                                                                                                      988   20    
                                                                          EVP_CIPHER_CTX_set_flags    995   3     
                                                                                                      997   3 

# Compute suspiciousness scores

In [5]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [6]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c', 'X509_VERIFY_PARAM_inherit')
fault idx: ('/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c', 'X509_VERIFY_PARAM_inherit', 212)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit, 212)",1,0,231,1,-0.004292,0.0,98824.0,172596.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/ssl_local.h, ssl_has_cert, 2355)",0,1,232,0,1.0,1.0,1718.0,1718.0
1,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_shutdown, 4404)",0,1,232,0,1.0,1.0,1718.0,1718.0
2,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_choose_cipher, 4276)",0,1,232,0,1.0,1.0,1718.0,1718.0
3,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_choose_cipher, 4278)",0,1,232,0,1.0,1.0,1718.0,1718.0
4,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_choose_cipher, 4280)",0,1,232,0,1.0,1.0,1718.0,1718.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_get_depth, 484)",0,1,232,0,1.0,1.0,1718.0,1718.0
1,"(/home/workspace/crypto/srp/libcrypto-shlib-srp_vfy.gcno//crypto/srp/srp_vfy.c, SRP_create_verifier_ex, 679)",0,1,232,0,1.0,1.0,1718.0,1718.0
2,"(/home/workspace/crypto/srp/libcrypto-shlib-srp_vfy.gcno//crypto/srp/srp_vfy.c, SRP_create_verifier_ex, 684)",0,1,232,0,1.0,1.0,1718.0,1718.0
3,"(/home/workspace/crypto/srp/libcrypto-shlib-srp_vfy.gcno//crypto/srp/srp_vfy.c, SRP_create_verifier_ex, 683)",0,1,232,0,1.0,1.0,1718.0,1718.0
4,"(/home/workspace/crypto/srp/libcrypto-shlib-srp_vfy.gcno//crypto/srp/srp_vfy.c, SRP_create_verifier_ex, 682)",0,1,232,0,1.0,1.0,1718.0,1718.0


# Method level SBFL

In [7]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,31,6,0,226,1,-0.025751,0.000000,145352.0,172596.0,/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c,EC_KEY_new,"(/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new)"
1,33,6,0,226,1,-0.025751,0.000000,145352.0,172596.0,/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c,EC_KEY_new,"(/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new)"
2,37,12,0,220,1,-0.051502,0.000000,157004.0,172596.0,/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c,EC_KEY_new_ex,"(/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_ex)"
3,39,12,0,220,1,-0.051502,0.000000,157004.0,172596.0,/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c,EC_KEY_new_ex,"(/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_ex)"
4,42,3,0,229,1,-0.012876,0.000000,127396.0,172596.0,/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c,EC_KEY_new_by_curve_name_ex,"(/home/workspace/crypto/ec/libcrypto-lib-ec_key.gcno//crypto/ec/ec_key.c, EC_KEY_new_by_curve_name_ex)"
...,...,...,...,...,...,...,...,...,...,...,...,...
172591,987,20,1,212,0,0.914163,0.218218,26387.0,26387.0,/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c,EVP_MD_CTX_clear_flags,"(/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_MD_CTX_clear_flags)"
172592,988,20,1,212,0,0.914163,0.218218,26387.0,26387.0,/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c,EVP_MD_CTX_clear_flags,"(/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_MD_CTX_clear_flags)"
172593,995,3,0,229,1,-0.012876,0.000000,127396.0,172596.0,/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c,EVP_CIPHER_CTX_set_flags,"(/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_CIPHER_CTX_set_flags)"
172594,997,3,0,229,1,-0.012876,0.000000,127396.0,172596.0,/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c,EVP_CIPHER_CTX_set_flags,"(/home/workspace/crypto/evp/libfips-lib-evp_lib.gcno//crypto/evp/evp_lib.c, EVP_CIPHER_CTX_set_flags)"


## Max aggregation

In [8]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",170,37,1,195,0,0.841202,3593.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_process_end_of_early_data)",4070,0,1,232,0,1.0,269.0
1,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_post_process_client_hello)",2260,0,1,232,0,1.0,269.0
2,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_handle_status_request)",2092,0,1,232,0,1.0,269.0
3,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_early_post_process_client_hello)",1608,0,1,232,0,1.0,269.0
4,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_server_key_exchange)",2471,0,1,232,0,1.0,269.0
5,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_server_hello)",2339,0,1,232,0,1.0,269.0
6,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_new_session_ticket)",3980,0,1,232,0,1.0,269.0
7,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, ossl_statem_server13_write_transition)",531,0,1,232,0,1.0,269.0
8,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, construct_stateless_ticket)",3763,0,1,232,0,1.0,269.0
9,"(/home/workspace/ssl/statem/libssl-shlib-statem_lib.gcno//ssl/statem/statem_lib.c, tls_setup_handshake)",127,0,1,232,0,1.0,269.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",170,37,1,195,0,0.162221,3593.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_process_end_of_early_data)",4070,0,1,232,0,1.0,269.0
1,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_post_process_client_hello)",2260,0,1,232,0,1.0,269.0
2,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_handle_status_request)",2092,0,1,232,0,1.0,269.0
3,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_early_post_process_client_hello)",1608,0,1,232,0,1.0,269.0
4,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_server_key_exchange)",2471,0,1,232,0,1.0,269.0
5,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_server_hello)",2339,0,1,232,0,1.0,269.0
6,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, tls_construct_new_session_ticket)",3980,0,1,232,0,1.0,269.0
7,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, ossl_statem_server13_write_transition)",531,0,1,232,0,1.0,269.0
8,"(/home/workspace/ssl/statem/libssl-shlib-statem_srvr.gcno//ssl/statem/statem_srvr.c, construct_stateless_ticket)",3763,0,1,232,0,1.0,269.0
9,"(/home/workspace/ssl/statem/libssl-shlib-statem_lib.gcno//ssl/statem/statem_lib.c, tls_setup_handshake)",127,0,1,232,0,1.0,269.0


## Line2Method voting (weight: sbfl score / dense rank)

In [9]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",0.301855,2151.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,170,37,1,195,0,0.841202,0.162221,36467.0,36467.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",38.0,0.022137
1,226,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
2,221,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
3,216,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
4,204,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
5,197,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
6,195,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
7,192,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
8,188,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172
9,187,60,1,172,0,0.742489,0.128037,42046.0,42046.0,/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c,X509_VERIFY_PARAM_inherit,"(/home/workspace/crypto/x509/libcrypto-shlib-x509_vpm.gcno//crypto/x509/x509_vpm.c, X509_VERIFY_PARAM_inherit)",61.0,0.012172


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,3705,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",1.0,1.0
1,3704,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",1.0,1.0
2,3703,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",1.0,1.0
3,3700,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",1.0,1.0
4,3699,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,3557,1,0,231,1,-0.004292,0.0,98824.0,172596.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",163.0,0.0
115,3545,3,0,229,1,-0.012876,0.0,127396.0,172596.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",163.0,0.0
116,3434,22,0,210,1,-0.094421,0.0,164049.0,172596.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",163.0,0.0
117,3433,22,0,210,1,-0.094421,0.0,164049.0,172596.0,/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c,ssl3_ctrl,"(/home/workspace/ssl/libssl-shlib-s3_lib.gcno//ssl/s3_lib.c, ssl3_ctrl)",163.0,0.0


Top 5:


,meth,vote,rank
0,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",47.394263,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,866,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",1.0,1.000
1,864,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",1.0,1.000
2,863,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",1.0,1.000
3,860,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",1.0,1.000
4,859,0,1,232,0,1.000000,1.0,1718.0,1718.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",1.0,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,745,24,1,208,0,0.896996,0.2,29114.0,29114.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",25.0,0.008
67,744,24,1,208,0,0.896996,0.2,29114.0,29114.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",25.0,0.008
68,743,24,1,208,0,0.896996,0.2,29114.0,29114.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",25.0,0.008
69,742,24,1,208,0,0.896996,0.2,29114.0,29114.0,/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c,tls_construct_ctos_early_data,"(/home/workspace/ssl/statem/libssl-shlib-extensions_clnt.gcno//ssl/statem/extensions_clnt.c, tls_construct_ctos_early_data)",25.0,0.008


# 감상

단순히 passing test에 의해서 한 번도 실행되지 않고 failing test에 의해서 실행되는 method가 너무 많다 (sbfl max agg top max rank: 269)